In [ ]:
!pip install openai==0.28.1

In [ ]:
import pandas as pd
import json
import openai

In [ ]:
prompt = 'A model that takes a paragraph summarizing a movie as input and, based on the summary, outputs the movie genre as one of the following: "action," "family," "romance," "crime," or "fantasy."'
temperature = 0.4
ANTHROPIC_API_KEY = "api key"
openai.api_key = ANTHROPIC_API_KEY

In [ ]:
df = pd.read_csv("/content/movie_category.csv")
df.head(10)

,prompt,response
0,"In a dystopian future, a skilled warrior named...",action
1,The movie tells the story of a young girl name...,family
2,The film is set in the 19th century and revolv...,romance
3,The story revolves around a group of thieves l...,crime
4,The movie follows the journey of a young boy n...,fantasy
5,"In this film, a group of interstellar outlaws,...",action
6,The movie follows the story of a young lion pr...,family
7,The film is centered around a young woman name...,romance
8,The film follows the life of a notorious mobst...,crime
9,"In this movie, a young hobbit named Frodo is c...",fantasy


In [ ]:
df.shape

(68, 2)

In [ ]:
df_train = df.sample(frac=0.8, random_state=42)
df_test = df.drop(df_train.index)

In [ ]:
df_test["prompt"].tolist()[0]

"The movie follows the story of a former Green Beret, John Rambo, who is haunted by memories of Vietnam. When his old commanding officer informs him that his unit has been killed by a deadly disease, Rambo embarks on a solo rescue mission. The film is filled with intense combat scenes, daring escapes, and a deep exploration of the effects of war on a soldier's psyche."

In [ ]:

def generate_system_message(prompt):

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\" SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)
print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

In [ ]:
system_message = generate_system_message(prompt)
print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given a summary of a movie, determine its genre as either "action," "family," "romance," "crime," or "fantasy."`. Feel free to re-run this cell if you want a better result.


In [ ]:
training_examples = []

# Create training examples in the format required for GPT-3.5 fine-tuning
for index, row in df_train.iterrows():
    training_example = {
        "messages": [
            {"role": "system", "content": system_message.strip()},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['response']}
        ]
    }
    training_examples.append(training_example)

# Save training examples to a .jsonl file
with open('training_examples.jsonl', 'w') as f:
    for example in training_examples:
        f.write(json.dumps(example) + '\n')

In [ ]:
inference_examples = []

# Create training examples in the format required for GPT-3.5 fine-tuning
for index, row in df_test.iterrows():
    inference_example = {
        "messages": [
            {"role": "system", "content": system_message.strip()},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['response']}
        ]
    }
    inference_examples.append(inference_example)

# Save inference examples to a .jsonl file
with open('inference_examples.jsonl', 'w') as f:
    for example in inference_examples:
        f.write(json.dumps(example) + '\n')

# **Upload the file to OpenAI**

In [ ]:
file_id = openai.File.create(
  file=open("/content/training_examples.jsonl", "rb"),
  purpose='fine-tune'
).id


In [ ]:
job = openai.FineTuningJob.create(training_file=file_id, model="gpt-3.5-turbo")
job_id = job.id

In [ ]:
job_id

'ftjob-XJJ2sNh5jbeQwQMgZQ2y3tS2'

In [ ]:
openai.FineTuningJob.list_events(id=job_id, limit=5)

<OpenAIObject list at 0x7a4ee717f1a0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-oXJkEKvOtl6b3L1p7LdhJxOT",
      "created_at": 1730219234,
      "level": "info",
      "message": "The job has successfully completed",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-nM7Gehw7MlzhdyMzUGcUCt79",
      "created_at": 1730219230,
      "level": "info",
      "message": "New fine-tuned model created",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-3hnhTijXSI7vheAdcYNoSMJH",
      "created_at": 1730219230,
      "level": "info",
      "message": "Checkpoint created at step 108",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-ZdqrDF8DtDpezN0C8daMuKV5",
      "created_at": 1730219230,
      "level": "info",
      "message": "Ch

In [ ]:
model_name_pre_object = openai.FineTuningJob.retrieve(job_id)
model_name = model_name_pre_object.fine_tuned_model
print(model_name)

ft:gpt-3.5-turbo-0125:personal::ANjLh7mp


In [ ]:
def geneate_response(question):
  response = openai.ChatCompletion.create(
    model=model_name,
    messages=[
      {
        "role": "system",
        "content": system_message,
      },
      {
          "role": "user",
          "content": question,
      }
    ],
  )
  return response.choices[0].message['content']

In [ ]:
geneate_response(df_test["prompt"].tolist()[0])

'action'

In [ ]:
df_test["prompt"].tolist()[0]

"The movie follows the story of a former Green Beret, John Rambo, who is haunted by memories of Vietnam. When his old commanding officer informs him that his unit has been killed by a deadly disease, Rambo embarks on a solo rescue mission. The film is filled with intense combat scenes, daring escapes, and a deep exploration of the effects of war on a soldier's psyche."

In [ ]:
df_test["response"].tolist()[0]

'action'

In [ ]:
df_test.shape

(14, 2)

In [ ]:
predicted = []
ground_truth = []
for i in range(14):
  predicted_response = geneate_response(df_test["prompt"].tolist()[i])
  predicted.append(predicted_response)
  ground_truth_response = df_test["response"].tolist()[i]
  ground_truth.append(ground_truth_response)
  print(predicted_response)
  print(ground_truth_response)


action
action
fantasy
fantasy
action
action
fantasy
fantasy
romance
romance
fantasy
fantasy
crime
crime
romance
romance
family
family
romance
romance
family
romance
romance
romance
romance
romance
crime
crime


In [ ]:
result_df = pd.DataFrame({'movie_summary':df_test["prompt"].tolist(), 'Ground Truth': ground_truth, 'Predicted': predicted})
result_df

,movie_summary,Ground Truth,Predicted
0,The movie follows the story of a former Green ...,action,action
1,"In this film, a young wizard named Harry Potte...",fantasy,fantasy
2,The movie is about a group of astronauts who e...,action,action
3,The film is set in a world where magic is real...,fantasy,fantasy
4,The film tells the story of a young woman name...,romance,romance
5,The film follows the journey of a hobbit named...,fantasy,fantasy
6,The movie follows the life of a notorious mobs...,crime,crime
7,The film revolves around a young woman named A...,romance,romance
8,The film follows the life of a young boy named...,family,family
9,The film revolves around a young woman named B...,romance,romance
